In [1]:
from pyspark.sql import SparkSession
from operator import add
import time

In [2]:
spark_session = SparkSession\
        .builder\
        .master("spark://host-192-168-1-37-ldsa:7077") \
        .appName("project_team25_thursday")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

In [ ]:
import pyspark.sql.functions as f
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

def convTime2Year(data):
    newyear = data[6:10]
    return int(newyear)

udf_toyear = udf(convTime2Year, IntegerType())

In [ ]:
# Load the csv from HDFS
data_frame = spark_session.read\
    .option("header", "true")\
    .csv('hdfs://192.168.1.153:9000/team25/libraryDataset/Library_Collection_Inventory.csv')
req_df = data_frame.select('BibNum','Title','Author','PublicationYear','Publisher','Subjects').distinct() 
data_frame_new = spark_session.read\
    .option("header", "true")\
    .csv('hdfs://192.168.1.153:9000/team25/libraryDataset/Checkouts_By_Title_Data_Lens_*.csv')
    # Load the csv from HDFS
data_frame_latest = spark_session.read\
    .option("header", "true")\
    .csv('hdfs://192.168.1.153:9000/team25/libraryDataset/Integrated_Library_System__ILS__Data_Dictionary.csv')\
    .cache()
data_frame_merged = data_frame_new.join(data_frame_latest, data_frame_new.ItemType == data_frame_latest.Code)
merged_data_frame = data_frame_merged.join(req_df, data_frame_merged.BibNumber == req_df.BibNum)
newdata = merged_data_frame.withColumn('Year', udf_toyear('CheckoutDateTime'))
start_time = time.time()
mostDesc = newdata.groupby('Description').count().select('Description', f.col('count')).orderBy('count',ascending=False).cache()
mostAuthorBook = newdata.filter('`Format Subgroup` == "Book" ').groupby('Author','Format Subgroup').count().select('Author','Format Subgroup', f.col('count')).orderBy('count',ascending=False).cache()
mostSeuessYear = newdata.filter('`Author` == "Seuss, Dr." ').groupby('Author','Year').count().select('Author','Year', f.col('count')).orderBy('Year',ascending=False).cache()
print(mostAuthorBook.show(),'mostAuthor')
print(mostDesc.show(),'MostDesc')
print(mostSeuessYear.show(),'MostSuess')
print(time.time() - start_time)

In [10]:
newdata.rdd.getNumPartitions()

200

In [11]:
n1 = newdata.union(newdata)
print(n1.rdd.getNumPartitions())
start_time = time.time()
mostDesc = n1.groupby('Description').count().select('Description', f.col('count')).orderBy('count',ascending=False).cache()
mostAuthorBook = n1.filter('`Format Subgroup` == "Book" ').groupby('Author','Format Subgroup').count().select('Author','Format Subgroup', f.col('count')).orderBy('count',ascending=False).cache()
mostSeuessYear = n1.filter('`Author` == "Seuss, Dr." ').groupby('Author','Year').count().select('Author','Year', f.col('count')).orderBy('Year',ascending=False).cache()
print(mostAuthorBook.show(),'mostAuthor')
print(mostDesc.show(),'MostDesc')
print(mostSeuessYear.show(),'MostSuess')
print(time.time() - start_time)

400
+--------------------+---------------+-------+
|              Author|Format Subgroup|  count|
+--------------------+---------------+-------+
|                null|           Book|2958118|
|          Seuss, Dr.|           Book| 435222|
|         Willems, Mo|           Book| 391558|
|      Meadows, Daisy|           Book| 357044|
|  Osborne, Mary Pope|           Book| 323880|
|     Rylant, Cynthia|           Book| 308842|
|Davis, Jim, 1945 ...|           Book| 289214|
|   Stilton, Geronimo|           Book| 259492|
|Patterson, James,...|           Book| 181184|
|Berenstain, Stan,...|           Book| 180194|
|   Holm, Jennifer L.|           Book| 159978|
|        Arnold, Tedd|           Book| 153380|
|  Pilkey, Dav, 1966-|           Book| 150360|
|      O'Connor, Jane|           Book| 142862|
|        Hunter, Erin|           Book| 140874|
|         Carle, Eric|           Book| 128142|
|       Park, Barbara|           Book| 117124|
|       Riordan, Rick|           Book| 115646|
|McCall S

In [12]:
n2 = n1.union(newdata)
print(n2.rdd.getNumPartitions())
start_time = time.time()
mostDesc = n2.groupby('Description').count().select('Description', f.col('count')).orderBy('count',ascending=False).cache()
mostAuthorBook = n2.filter('`Format Subgroup` == "Book" ').groupby('Author','Format Subgroup').count().select('Author','Format Subgroup', f.col('count')).orderBy('count',ascending=False).cache()
mostSeuessYear = n2.filter('`Author` == "Seuss, Dr." ').groupby('Author','Year').count().select('Author','Year', f.col('count')).orderBy('Year',ascending=False).cache()
print(mostAuthorBook.show(),'mostAuthor')
print(mostDesc.show(),'MostDesc')
print(mostSeuessYear.show(),'MostSuess')
print(time.time() - start_time)

600


Py4JJavaError: An error occurred while calling o568.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 51 in stage 247.0 failed 4 times, most recent failure: Lost task 51.3 in stage 247.0 (TID 9651, 192.168.1.44, executor 2): java.io.IOException: No space left on device
	at java.io.FileOutputStream.writeBytes(Native Method)
	at java.io.FileOutputStream.write(FileOutputStream.java:326)
	at org.apache.spark.storage.TimeTrackingOutputStream.write(TimeTrackingOutputStream.java:58)
	at java.io.BufferedOutputStream.flushBuffer(BufferedOutputStream.java:82)
	at java.io.BufferedOutputStream.flush(BufferedOutputStream.java:140)
	at net.jpountz.lz4.LZ4BlockOutputStream.flush(LZ4BlockOutputStream.java:240)
	at java.io.BufferedOutputStream.flush(BufferedOutputStream.java:141)
	at java.io.DataOutputStream.flush(DataOutputStream.java:123)
	at org.apache.spark.sql.execution.UnsafeRowSerializerInstance$$anon$1.flush(UnsafeRowSerializer.scala:91)
	at org.apache.spark.

In [3]:
# Load the csv from HDFS
data_frame = spark_session.read\
    .option("header", "true")\
    .csv('hdfs://192.168.1.153:9000/team25/libraryDataset/Library_Collection_Inventory.csv')

In [31]:
req_df = data_frame.select('BibNum','Title','Author','PublicationYear','Publisher','Subjects').distinct()

In [35]:
data_frame_new = spark_session.read\
    .option("header", "true")\
    .csv('hdfs://192.168.1.153:9000/team25/libraryDataset/Checkouts_By_Title_Data_Lens_*.csv')

In [40]:
# Load the csv from HDFS
data_frame_latest = spark_session.read\
    .option("header", "true")\
    .csv('hdfs://192.168.1.153:9000/team25/libraryDataset/Integrated_Library_System__ILS__Data_Dictionary.csv')\
    .cache()

In [45]:
data_frame_merged = data_frame_new.join(data_frame_latest, data_frame_new.ItemType == data_frame_latest.Code)

In [48]:
merged_data_frame = data_frame_merged.join(req_df, data_frame_merged.BibNumber == req_df.BibNum)

In [51]:
merged_data_frame.rdd.getNumPartitions()

200

In [82]:
import time

In [77]:
newdata = merged_data_frame.withColumn('Year', udf_toyear('CheckoutDateTime'))

In [79]:
newdata.select('Year').first()

Row(Year=2005)

In [83]:
start_time = time.time()
merged_data_frame.select('CheckoutDateTime').first()
print(time.time() - start_time)

139.55632281303406


In [56]:
mostDesc = merged_data_frame.groupby('Description').count().select('Description', f.col('count')).orderBy('count',ascending=False).cache()

In [58]:
mostAuthorBook = merged_data_frame.filter('`Format Subgroup` == "Book" ').groupby('Author','Format Subgroup').count().select('Author','Format Subgroup', f.col('count')).orderBy('count',ascending=False).cache()

In [59]:
mostAuthorBook.show()

+--------------------+---------------+-------+
|              Author|Format Subgroup|  count|
+--------------------+---------------+-------+
|                null|           Book|1479059|
|          Seuss, Dr.|           Book| 217611|
|         Willems, Mo|           Book| 195779|
|      Meadows, Daisy|           Book| 178522|
|  Osborne, Mary Pope|           Book| 161940|
|     Rylant, Cynthia|           Book| 154421|
|Davis, Jim, 1945 ...|           Book| 144607|
|   Stilton, Geronimo|           Book| 129746|
|Patterson, James,...|           Book|  90592|
|Berenstain, Stan,...|           Book|  90097|
|   Holm, Jennifer L.|           Book|  79989|
|        Arnold, Tedd|           Book|  76690|
|  Pilkey, Dav, 1966-|           Book|  75180|
|      O'Connor, Jane|           Book|  71431|
|        Hunter, Erin|           Book|  70437|
|         Carle, Eric|           Book|  64071|
|       Park, Barbara|           Book|  58562|
|       Riordan, Rick|           Book|  57823|
|McCall Smith

In [60]:
mostDesc.show()

+--------------------+--------+
|         Description|   count|
+--------------------+--------+
|      Book: Adult/YA|20076679|
|       DVD: Adult/YA|15881200|
|           Book: Juv|15840463|
|        CD: Adult/YA| 7423766|
|DVD: Juv Circulating| 2284498|
|             CD: Juv|  812341|
|Music Score: Adul...|  112357|
|Book: Adult/Forme...|   29337|
|            Kit: Juv|   27130|
|     Peak Picks Book|   27093|
| Equipment: Adult/YA|   17572|
|  Book: Ref Adult/YA|   11775|
|Laptop: Hourly Ci...|    6359|
|  Uncataloged Folder|    5957|
|DVD: Adult/Former...|    4230|
|Song Book: Bound Juv|    1224|
|Equipment: Adult/...|     887|
|       Kit: Adult/YA|     393|
|     Tablet computer|     386|
|Equipment: Ref Ad...|     338|
+--------------------+--------+
only showing top 20 rows



In [85]:
mostSeuessYear = newdata.filter('`Author` == "Seuss, Dr." ').groupby('Author','Year').count().select('Author','Year', f.col('count')).orderBy('Year',ascending=False).cache()

In [87]:
mostSeuessYear.show()

+----------+----+-----+
|    Author|Year|count|
+----------+----+-----+
|Seuss, Dr.|2017|17029|
|Seuss, Dr.|2016|19137|
|Seuss, Dr.|2015|22353|
|Seuss, Dr.|2014|21170|
|Seuss, Dr.|2013|23796|
|Seuss, Dr.|2012|25102|
|Seuss, Dr.|2011|16706|
|Seuss, Dr.|2010|17019|
|Seuss, Dr.|2009|15387|
|Seuss, Dr.|2008|14722|
|Seuss, Dr.|2007|14667|
|Seuss, Dr.|2006|13695|
|Seuss, Dr.|2005| 9182|
+----------+----+-----+



In [94]:
import pyspark

In [95]:
newdata.persist(pyspark.StorageLevel.MEMORY_ONLY)

DataFrame[BibNumber: string, ItemBarcode: string, ItemType: string, Collection: string, CallNumber: string, CheckoutDateTime: string, Code: string, Description: string, Code Type: string, Format Group: string, Format Subgroup: string, Category Group: string, Category Subgroup: string, BibNum: string, Title: string, Author: string, PublicationYear: string, Publisher: string, Subjects: string, Year: int]

In [96]:
newdata.count()

62566971

In [108]:
sizedf = newdata.filter('`Description` == "Uncataloged Folder"')

In [112]:
sizedf.count()

KeyboardInterrupt: 

In [ ]:
sizedf.rdd.getNumPartitions()

In [111]:
sizedf.write.csv('hdfs://192.168.1.153:9000/team25/libraryDataset/temp.csv')

KeyboardInterrupt: 